# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papatowai,-46.5619,169.4708,11.62,53,93,1.57,NZ,1717467391
1,1,ishigaki,24.3448,124.1572,28.07,65,75,6.17,JP,1717467391
2,2,punta arenas,-53.1500,-70.9167,0.06,93,0,2.57,CL,1717467116
3,3,nikolayevsk-on-amure,53.1406,140.7300,5.12,67,100,3.58,RU,1717467391
4,4,port-aux-francais,-49.3500,70.2167,1.56,91,100,17.59,TF,1717467391


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.scatter(x='Lng', y='Lat', c='Humidity', size='Humidity', 
                                       hover_cols=['City', 'Humidity'], colorbar=True,
                                       title='City Map with Humidity')

# Display the map
map_plot

:Scatter   [Lng]   (Lat,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df[(city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 25) & (city_data_df['Humidity'] < 50)].copy()

# Drop any rows with null values
criteria_df.dropna(inplace=True)

# Display sample data
criteria_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,saynshand,44.8824,110.1163,24.55,31,90,4.44,MN,1717467393
52,52,college,64.8569,-147.8028,20.97,44,75,3.09,US,1717467397
56,56,mount isa,-20.7333,139.5000,21.87,33,40,4.63,AU,1717467398
179,179,kuqa,41.7278,82.9364,20.20,21,100,2.01,CN,1717467412
191,191,badger,64.8000,-147.5333,20.93,39,20,3.09,US,1717467414


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = criteria_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
20,saynshand,MN,44.8824,110.1163,31,
52,college,US,64.8569,-147.8028,44,
56,mount isa,AU,-20.7333,139.5000,33,
179,kuqa,CN,41.7278,82.9364,21,
191,badger,US,64.8000,-147.5333,39,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 5000  # Adjust the radius as needed
params = {
    "type": "hotel",
    "radius": radius,
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lat},{lng}"
    params["bias"] = f"point:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
saynshand - nearest hotel: No hotel found
college - nearest hotel: No hotel found
mount isa - nearest hotel: No hotel found
kuqa - nearest hotel: No hotel found
badger - nearest hotel: No hotel found
jiuquan - nearest hotel: No hotel found
karratha - nearest hotel: No hotel found
vilyuysk - nearest hotel: No hotel found
verkhnevilyuysk - nearest hotel: No hotel found
moranbah - nearest hotel: No hotel found
al bayda' - nearest hotel: No hotel found
undurkhaan - nearest hotel: No hotel found
hibbing - nearest hotel: No hotel found
oyama - nearest hotel: No hotel found
nuevo ideal - nearest hotel: No hotel found
metlili chaamba - nearest hotel: No hotel found
biloela - nearest hotel: No hotel found
bongaree - nearest hotel: No hotel found
toliara - nearest hotel: No hotel found
genhe - nearest hotel: No hotel found
embi - nearest hotel: No hotel found
yima - nearest hotel: No hotel found
kashgar - nearest hotel: No hotel found
port hedland - nearest hotel: No hotel 

,City,Country,Lat,Lng,Humidity,Hotel Name
20,saynshand,MN,44.8824,110.1163,31,No hotel found
52,college,US,64.8569,-147.8028,44,No hotel found
56,mount isa,AU,-20.7333,139.5000,33,No hotel found
179,kuqa,CN,41.7278,82.9364,21,No hotel found
191,badger,US,64.8000,-147.5333,39,No hotel found
194,jiuquan,CN,39.7432,98.5174,21,No hotel found
217,karratha,AU,-20.7377,116.8463,40,No hotel found
231,vilyuysk,RU,63.7553,121.6247,40,No hotel found
238,verkhnevilyuysk,RU,63.4453,120.3167,39,No hotel found
241,moranbah,AU,-22.0016,148.0466,28,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.scatter(x='Lng', y='Lat', c='Humidity', size='Humidity', 
                                   hover_cols=['City', 'Country', 'Hotel Name'], 
                                   colorbar=True, title='City Map with Humidity and Nearest Hotel')

# Display the map
map_plot

:Scatter   [Lng]   (Lat,Humidity,City,Country,Hotel Name)